In [1]:
import requests
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc

In [10]:
def temp_city(ciudad):
    # Iniciar driver
    Browser = uc.Chrome()
    url = 'https://www.tiempo.es/'
    Browser.get(url)
    time.sleep(3)
    # Navegacion
    try:
        Browser.find_element('xpath','//*[@id="cmp-btn-accept"]').click()   
    except:
        pass
    Browser.find_element('xpath','//*[@id="search_box"]').send_keys(f'{ciudad}')
    Browser.find_element('xpath','//*[@id="submitSearchButton"]').click()
    try:
        Browser.find_element('xpath','//*[@id="cmp-btn-accept"]').click()   
    except:
        pass
    Browser.find_element('xpath','//*[@class="res"]').click()
    try:
        Browser.find_element('xpath','//*[@id="cmp-btn-accept"]').click()   
    except:
        pass
    #Parseo
    html = Browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    #Obtencion de datos
    temp_actual = int(soup.find('div',{'class':'tmp'}).text.replace('º',''))
    contenedor = soup.find('div',{'id':'mainPrediction'}).find('div',{'class':'info'}).find_all('div',{'class':'item'})
    temp_maxima = (contenedor[0].text.rstrip())
    temp_maxima = int(re.findall(r'\d+',temp_maxima)[0])
    lluvia = int(contenedor[1].text.strip().replace('Lluvia',"").replace('%',''))
    viento = (contenedor[2].text.strip().replace('km/h','').rstrip())
    viento = int(re.findall(r'\d+',viento)[0])
    
    Browser.close()
    Browser.quit()
    return temp_actual,temp_maxima,lluvia,viento

In [11]:
# Creacion de dataframe
df = pd.DataFrame(columns=(['Ciudad','Temp Actual °C','Temp Maximo °C','Prob Lluvia','Viento Km/h']))

In [12]:
ciudad = input("Ingrese el nombre de la ciudad a buscar:").capitalize()
resultado = temp_city(ciudad)

Ingrese el nombre de la ciudad a buscar: barcelona


In [13]:
#Agregar datos a dataframe
temp_actual = resultado[0]
temp_maxima = resultado[1]
lluvia = resultado[2]
viento = resultado[3]
df = df._append({'Ciudad':ciudad,'Temp Actual °C':temp_actual,'Temp Maximo °C':temp_maxima,'Prob Lluvia':lluvia,'Viento Km/h':viento}, ignore_index=True)
df

,Ciudad,Temp Actual °C,Temp Maximo °C,Prob Lluvia,Viento Km/h
0,Barcelona,19,21,43,30
